In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
import os
import pandas as pd

<h1> Load Checkerboard Data </h1>

In [ ]:
image_points_path = './image_points.mat'
pairs_used_path = './pairs_used.mat'
image_points = scipy.io.loadmat(image_points_path)['imagePoints']
pairs_used = np.squeeze(scipy.io.loadmat(pairs_used_path)['pairsUsed'])

In [ ]:
left_images_folder = './rectified_filtered_left_calibration_images'
right_images_folder = './rectified_filtered_right_calibration_images'

left_image_paths = [os.path.join(left_images_folder, f) for f in sorted(os.listdir(left_images_folder))]
right_image_paths = [os.path.join(right_images_folder, f) for f in sorted(os.listdir(right_images_folder))]
left_image_paths = [f for i, f in enumerate(left_image_paths) if pairs_used[i] == 1]
right_image_paths = [f for i, f in enumerate(right_image_paths) if pairs_used[i] == 1]


<h1> Get predicted world points for all checkerboard points </h1>

<h3> Define camera attributes and helper functions </h3>

In [ ]:
# all distance are in meters
FOCAL_LENGTH = 0.008406732
BASELINE = 0.1266040474
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000
CHECKERBOARD_SIDE_LENGTH = 0.052441185



In [ ]:
def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / PIXEL_COUNT_WIDTH)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / PIXEL_COUNT_HEIGHT)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])

def depth_from_disp(disp):
    """ calculate the depth of the point based on the disparity value """
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth

<h3> Generate world point predictions </h3>

In [ ]:
world_points = np.zeros([image_points.shape[0], 3, image_points.shape[2]])
disparities = np.zeros([image_points.shape[0], image_points.shape[2]])

num_stereo_image_pairs = image_points.shape[2]
num_checkerboard_points = image_points.shape[0]

for i in range(num_stereo_image_pairs):
    left_image_checkerboard_points = image_points[:,:,i,0]
    right_image_checkerboard_points = image_points[:,:,i,1]
    disparity_values = left_image_checkerboard_points - right_image_checkerboard_points
    horizontal_disparity_values = disparity_values[:, 0]
    for j, point in enumerate(left_image_checkerboard_points):
        depth = depth_from_disp(horizontal_disparity_values[j])
        x, y, z = convert_to_world_point(point[0], point[1], depth)
        world_points[j, :, i] = np.array([x, y, z])
        disparities[j, i] = horizontal_disparity_values[j]
    
    


<h3> Get predicted distances vs. ground truth distances </h3>

In [ ]:
# all distances in meters

CHECKERBOARD_WIDTH = 18
CHECKERBOARD_HEIGHT = 11
SQUARE_SIDE_LENGTH = 0.052441185
def get_ground_truth_distance(m, n):
    col1 = int(m / CHECKERBOARD_HEIGHT)
    row1 = m % CHECKERBOARD_HEIGHT
    
    col2 = int(n / CHECKERBOARD_HEIGHT)
    row2 = n % CHECKERBOARD_HEIGHT
    
    ground_truth_distance = SQUARE_SIDE_LENGTH * ((col2 - col1)**2 + (row2 - row1)**2)**0.5
    return ground_truth_distance
    
    
    

In [ ]:
point_1_disparities = []
point_2_disparities = []
ground_truth_distances = []
errors = []
relative_errors = []

point_1_left_x_coordinates = []
point_1_left_y_coordinates = []
point_2_left_x_coordinates = []
point_2_left_y_coordinates = []

point_1_right_x_coordinates = []
point_1_right_y_coordinates = []
point_2_right_x_coordinates = []
point_2_right_y_coordinates = []

stereo_image_pair_numbers = []
point_1_numbers = []
point_2_numbers = []

for i in range(num_stereo_image_pairs):
    for m in range(len(world_points[:, :, i])):
        for n in range(m+1, len(world_points[:, :, i])):
            world_point_1 = world_points[:, :, i][m]
            world_point_2 = world_points[:, :, i][n]
            predicted_distance = np.linalg.norm(world_point_1 - world_point_2)
            
            # get relative and ground truth differences
            ground_truth_distance = get_ground_truth_distance(m, n)
            error = predicted_distance - ground_truth_distance
            relative_error = error / ground_truth_distance
            errors.append(error)
            relative_errors.append(relative_error)
            ground_truth_distances.append(ground_truth_distance)
            
            # get horizontal disparity mean
            point_1_disparity = disparities[:, i][m]
            point_2_disparity = disparities[:, i][n]
            point_1_disparities.append(point_1_disparity)
            point_2_disparities.append(point_2_disparity)
            
            # get x and y image coordinate means for each point
            point_1_left_image_coordinates = image_points[m, :, i, 0]
            point_1_right_image_coordinates = image_points[m, :, i, 1]
            point_2_left_image_coordinates = image_points[n, :, i, 0]
            point_2_right_image_coordinates = image_points[n, :, i, 1]
            
            point_1_left_x_coordinate = point_1_left_image_coordinates[0]
            point_1_left_y_coordinate = point_1_left_image_coordinates[1]
            point_2_left_x_coordinate = point_2_left_image_coordinates[0]
            point_2_left_y_coordinate = point_2_left_image_coordinates[1]
            
            point_1_right_x_coordinate = point_1_right_image_coordinates[0]
            point_1_right_y_coordinate = point_1_right_image_coordinates[1]
            point_2_right_x_coordinate = point_2_right_image_coordinates[0]
            point_2_right_y_coordinate = point_2_right_image_coordinates[1]
            
            point_1_left_x_coordinates.append(point_1_left_x_coordinate)
            point_1_left_y_coordinates.append(point_1_left_y_coordinate)
            point_2_left_x_coordinates.append(point_2_left_x_coordinate)
            point_2_left_y_coordinates.append(point_2_left_y_coordinate)

            point_1_right_x_coordinates.append(point_1_right_x_coordinate)
            point_1_right_y_coordinates.append(point_1_right_y_coordinate)
            point_2_right_x_coordinates.append(point_2_right_x_coordinate)
            point_2_right_y_coordinates.append(point_2_right_y_coordinate)
            
            # add stereo image pair number, point 1 number, and point 2 number
            stereo_image_pair_numbers.append(i)
            point_1_numbers.append(m)
            point_2_numbers.append(n)


df = pd.DataFrame({
    'point_1_disparity': point_1_disparities,
    'point_2_disparity': point_2_disparities,
    'ground_truth_distance': ground_truth_distances,
    'error': errors,
    'relative_error': relative_errors,
    
    'point_1_left_x_coordinate': point_1_left_x_coordinates,
    'point_1_left_y_coordinate': point_1_left_y_coordinates,
    'point_2_left_x_coordinate': point_2_left_x_coordinates,
    'point_2_left_y_coordinate': point_2_left_y_coordinates,
    
    'point_1_right_x_coordinate': point_1_right_x_coordinates,
    'point_1_right_y_coordinate': point_1_right_y_coordinates,
    'point_2_right_x_coordinate': point_2_right_x_coordinates,
    'point_2_right_y_coordinate': point_2_right_y_coordinates,
    
    'stereo_image_pair_number': stereo_image_pair_numbers,
    'point_1_number': point_1_numbers,
    'point_2_number': point_2_numbers
})
            
            

In [ ]:
mask = (df.ground_truth_distance > 0.4) & (df.ground_truth_distance < 0.65)
df[mask].sort_values('ground_truth_distance', ascending=False)

In [ ]:
plt.hist(df[mask].error)
plt.show()

In [ ]:
def display_stereo_frame_pair(stereo_image_pair_number):
    left_image_points = image_points[:, :, stereo_image_pair_number, 0]
    right_image_points = image_points[:, :, stereo_image_pair_number, 1]
    
    left_im = plt.imread(left_image_paths[stereo_image_pair_number])
    right_im = plt.imread(right_image_paths[stereo_image_pair_number])
    plt.figure(figsize=(50, 50))
    plt.subplot(2, 1, 1)
    plt.imshow(left_im)
    plt.scatter(left_image_points[:, 0], left_image_points[:, 1], s=60, c='red', marker='o')
    plt.subplot(2, 1, 2)
    plt.imshow(right_im)
    plt.scatter(right_image_points[:, 0], right_image_points[:, 1], s=60, c='red', marker='o')
    plt.show()

In [ ]:
display_stereo_frame_pair(18)

In [ ]:
plt.hist(errors, 20)

In [ ]:
plt.scatter(ground_truth_distances, relative_errors)

In [ ]:
# plt.scatter(disparity_means, differences)
buckets = [400, 500, 600, 700, 800, 900]
bucket_size = 100
rel_diffs_by_bucket = {}
for bucket in buckets:
    left_endpoint, right_endpoint = bucket - 0.5 * bucket_size, bucket + 0.5 * bucket_size
    rel_diffs = relative_differences[np.logical_and(left_endpoint < disparity_means, disparity_means < right_endpoint)]
    rel_diffs_by_bucket[bucket] = list(rel_diffs)
    
    
    

In [ ]:
for bucket in buckets:
    print('Bucket: {}, Median rel diff: {}'.format(bucket, np.median(rel_diffs_by_bucket[bucket])))

In [ ]:
len(disparity_means)

In [ ]:
plt.scatter(disparity_means, relative_differences)
plt.show()

In [ ]:
differences[left_endpoint < disparity_means]

In [ ]:
len(differences)